**Use this cell for notes and comments - will delete before submission:**
## **- Pull before pushing to github!!!**
- Each group is expected to prepare a 1 page (max 500 words) written proposal that identifies the dataset they plan to work on, as well as the question they would like to answer using that dataset for their group project. The proposal should be done in a Jupyter notebook, and then submitted both as an .html file (File -> Download As -> HTML) and an .ipynb file that is reproducible (i.e. works and runs without any additional files.)
- will leave descriptions for each section for now as we write but let's delete before submitting (EG)
- Google doc: https://docs.google.com/document/d/14MA5buy1aAFNB8jjsdWar8QPMuju9pH_iFZn3Zmtx9Q/edit

# Title

Group 005-36: Elena Greenall, Noah Hynds, Dillon Li

### **Introduction**
insrt . 

### **Preliminary Exploratory Data Analysis**
- Demonstrate that the dataset can be read from the web into R
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis).
- An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data.
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [2]:
#the first step is to import the necessary libraries
library(tidyverse)
library(tidymodels)
library(RColorBrewer)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
#next the dataset can be loaded and tidied
smoking_df <- read_csv("data/smoking.csv") 

smoking_df

Rows: 55692 Columns: 27
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): gender, oral, tartar
dbl (24): ID, age, height(cm), weight(kg), waist(cm), eyesight(left), eyesig...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),⋯,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
0,F,40,155,60,81.3,1.2,1.0,1,1,⋯,12.9,1,0.7,18,19,27,Y,0,Y,0
1,F,40,160,60,81.0,0.8,0.6,1,1,⋯,12.7,1,0.6,22,19,18,Y,0,Y,0
2,M,55,170,60,80.0,0.8,0.8,1,1,⋯,15.8,1,1.0,21,16,22,Y,0,N,1
3,M,40,165,70,88.0,1.5,1.5,1,1,⋯,14.7,1,1.0,19,26,18,Y,0,Y,0
4,F,40,155,60,86.0,1.0,1.0,1,1,⋯,12.5,1,0.6,16,14,22,Y,0,N,0
5,M,30,180,75,85.0,1.2,1.2,1,1,⋯,16.2,1,1.2,18,27,33,Y,0,Y,0
6,M,40,160,60,85.5,1.0,1.0,1,1,⋯,17.0,1,0.7,21,27,39,Y,1,Y,1
7,M,45,165,90,96.0,1.2,1.0,1,1,⋯,15.0,1,1.3,38,71,111,Y,0,Y,0
9,F,50,150,60,85.0,0.7,0.8,1,1,⋯,13.7,1,0.8,31,31,14,Y,0,N,0


### **Methods**
- Explain how you will conduct either your data analysis and which variables/columns you will use.
- Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction
- Describe at least one way that you will visualize the results

### **Expected Outcomes and Significance**
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

### **References**